# Install dependencies


In [24]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
from matplotlib import pyplot as plt
import numpy as np

# Hyperparameters

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.0001
noise_dim = 64
img_dim = 28 * 28 * 1
batch_size = 32
num_epochs = 50

# Create Discriminator

In [26]:
class Discriminator(nn.Module):
  def __init__(self, input_features):
    super(Discriminator, self).__init__()

    self.disc = nn.Sequential(
        nn.Linear(input_features, 128),
        nn.LeakyReLU(0.1),
        nn.Linear(128, 1),
        nn.Sigmoid(),
    )

  def forward(self, x):
    return self.disc(x)



# Create Generator

In [27]:
class Generator(nn.Module):
  def __init__(self, noise_dim, img_dim):
    super(Generator, self).__init__()

    self.gen = nn.Sequential(
        nn.Linear(noise_dim, 256),
        nn.LeakyReLU(0.1),
        nn.Linear(256, img_dim),
        nn.Tanh(),
    )

  def forward(self, x):
    return self.gen(x)

# No comment

In [28]:
disc = Discriminator(img_dim).to(device)
gen = Generator(noise_dim, img_dim).to(device)

fixed_noise = torch.randn((batch_size, noise_dim)).to(device)

transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))]
)

# Handle Dataset

In [29]:
dataset = datasets.MNIST(root='dataset/', transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizer & Loss Function

In [30]:
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr)
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()

writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [1]:
from os import write
for epoch in range(num_epochs):
  for batch_id, (real, _) in enumerate(loader):
    real = real.view(-1, 784).to(device)
    batch_size = real.shape[0]

    ### Train Discriminator
    noise = torch.randn(batch_size, noise_dim).to(device)
    fake = gen(noise)
    disc_real = disc(real).view(-1)
    lossD_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake.detach()).view(-1) # Detach fake here
    lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake) / 2

    disc.zero_grad()
    lossD.backward()
    opt_disc.step()

    ### Train Generator
    output = disc(fake).view(-1)
    lossG = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

    if batch_id == 0:
      print(f"Epoch [{epoch}/{num_epochs}] Batch {batch_id}/{len(loader)} \
            Loss D: {lossD:.4f}, loss G: {lossG:.4f}")

      with torch.no_grad():
        fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
        data = real.reshape(-1, 1, 28, 28)
        img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
        img_grid_real = torchvision.utils.make_grid(data, normalize=True)

        writer_fake.add_image("Fake", img_grid_fake, global_step=step)
        writer_real.add_image("Real", img_grid_real, global_step=step)

        step += 1

NameError: name 'num_epochs' is not defined